In [ ]:
# 🚀 1. Importar librerías necesarias
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional # Añadido Bidirectional
# from tensorflow.keras import regularizers # No se usa L2 ahora
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
# from sklearn.utils.class_weight import compute_class_weight # No se usa

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import re
import unicodedata
import matplotlib.pyplot as plt
import os
import pickle
import gzip
import subprocess

In [ ]:
# 🚀 1b. Descargar los embeddings fastText en español (ACTIVADO)
embed_path = '/kaggle/working/cc.es.300.vec.gz'
embed_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz'
PRETRAINED_EMBEDDING_DIM = 300

if not os.path.exists(embed_path):
    print(f"Descargando embeddings fastText desde {embed_url}...")
    try:
        result = subprocess.run(['wget', '-O', embed_path, embed_url], check=True, capture_output=True, text=True)
        print("Descarga completada.")
    except Exception as e:
        print(f"Error al descargar embeddings: {e}")
        embed_path = None # Fallback
else:
   print(f"Embeddings fastText ya existen en {embed_path}.")
if not os.path.exists(embed_path): embed_path = None # Doble check
if embed_path: print("Embeddings pre-entrenados ACTIVADOS.")
else: print("Embeddings pre-entrenados DESACTIVADOS."); exit() # Salir si no hay embeddings

In [ ]:
# 🚀 1c. Cargar embeddings fastText (ACTIVADO)
embeddings_index = {}
num_loaded = 0
if embed_path and os.path.exists(embed_path):
    print(f"Cargando embeddings desde {embed_path}...")
    try:
        with gzip.open(embed_path, 'rt', encoding='utf-8', newline='\n', errors='ignore') as f:
            header = f.readline().split(); print(f"Cabecera: {header}")
            for line in f:
                values = line.rstrip().split(' ')
                word = values[0]
                if len(values) == PRETRAINED_EMBEDDING_DIM + 1:
                   try:
                       coefs = np.asarray(values[1:], dtype='float32')
                       embeddings_index[word] = coefs; num_loaded += 1
                   except ValueError: pass
    except Exception as e: print(f"Error al leer embeddings: {e}"); embeddings_index = {}
    print(f'Se cargaron {num_loaded} vectores.')
    if not embeddings_index: print("ADVERTENCIA: Falló carga de embeddings."); exit() # Salir si falló
else: print("Archivo de embeddings no encontrado."); exit() # Salir si no hay archivo

In [ ]:
# 🚀 2. Configurar uso de GPU si está disponible
# (Código igual)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e: print(e)
else: print("No se detectó GPU, se usará CPU.")

In [ ]:
# 🚀 3. Cargar tus datos
# (Código igual)
try:
  df = pd.read_csv('https://raw.githubusercontent.com/adiacla/sentimientos/refs/heads/main/emociones.csv', sep="|")
  print("Datos cargados:"); print(df.head()); print(f"\nForma inicial: {df.shape}")
except Exception as e: print(f"Error al cargar CSV: {e}"); exit()

In [ ]:
# 🚀 4. Preprocesamiento y Limpieza de Texto
# (Código igual)
emotion_translation = {
 'depressed': 'deprimido', 'hopeless': 'desesperanzado', 'lonely': 'solitario', 'suicidal': 'suicida',
 'disappointment': 'decepción', 'disgust': 'asco', 'remorse': 'remordimiento', 'grief': 'duelo',
 'embarrassment': 'vergüenza', 'fear': 'miedo', 'nervousness': 'nerviosismo', 'sadness': 'tristeza',
 'anger': 'enojo', 'neutral': 'neutral'
}
df['emotion_es'] = df['emotion'].map(emotion_translation)
print("\nEmociones traducidas:"); print(df['emotion_es'].value_counts())
def clean_text(text):
  if not isinstance(text, str): return ""
  text = text.lower()
  text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')
  text = re.sub(r"http\S+", "", text); text = re.sub(r"@\w+", "", text)
  text = re.sub(r"#\w+", "", text); text = re.sub(r"[^a-zA-Z\s]", "", text)
  text = re.sub(r'\s+', ' ', text).strip(); return text
df['tweet_clean'] = df['tweet'].apply(clean_text)
print("\nTweets limpios (head):"); print(df[['tweet_clean']].head())
df = df.dropna(subset=['tweet_clean', 'emotion_es'])
df = df[df['tweet_clean'].str.strip() != '']; print(f"\nForma tras limpieza: {df.shape}")

In [ ]:
# 🚀 5. Tokenizar los textos
# (Código igual)
oov_token = '<OOV>'; tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(df['tweet_clean'].values); word_index = tokenizer.word_index
vocab_size = len(word_index) + 1; print(f"\nTamaño del vocabulario: {vocab_size}")
sequences = tokenizer.texts_to_sequences(df['tweet_clean'].values)

In [ ]:
# 🚀 6. Padding de secuencias
# (Código igual)
maxlen = 50; X = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
print(f"\nForma de X tras padding: {X.shape}")

In [ ]:
# 🚀 7. Codificar las etiquetas
# (Código igual)
label_encoder = LabelEncoder(); integer_encoded = label_encoder.fit_transform(df['emotion_es'].values)
num_classes = len(label_encoder.classes_); print(f'\nNum clases: {num_classes}'); print(f'Clases: {label_encoder.classes_}')
y = to_categorical(integer_encoded, num_classes=num_classes); print(f"Forma de y (one-hot): {y.shape}")

In [ ]:
# 🚀 8. Dividir datos en entrenamiento y validación
# (Código igual)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nForma train: {X_train.shape}, {y_train.shape}"); print(f"Forma val: {X_val.shape}, {y_val.shape}")

In [ ]:
# 🚀 10. Crear Matriz de Embedding (ACTIVADO)
# (Código igual, asegura que use_pretrained=True si embeddings_index tiene datos)
embedding_matrix = None; use_pretrained = False
print("Creando matriz de embedding...")
embedding_matrix = np.zeros((vocab_size, PRETRAINED_EMBEDDING_DIM)); hits = 0; misses = 0
if embeddings_index:
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector; hits += 1
        else: misses += 1
    print(f"Palabras encontradas: {hits}"); print(f"Palabras no encontradas: {misses}")
    if hits > 0: use_pretrained = True; print("Se usarán embeddings pre-entrenados.")
    else: print("ADVERTENCIA: Ninguna palabra encontrada."); embedding_matrix = None; use_pretrained = False
else: print("No se usarán embeddings pre-entrenados."); embedding_matrix = None
embedding_dim_model = PRETRAINED_EMBEDDING_DIM if use_pretrained else 128
print(f"Dimensión de embedding a usar: {embedding_dim_model}")
if not use_pretrained: print("ADVERTENCIA: Fallback a embeddings entrenables."); embedding_dim_model = 128 # Asegurar fallback

In [ ]:
# 🚀 11. Construir el modelo LSTM (PARA FASE 1 - Embeddings Congelados)

hidden_units = 64; dense_units = 32; dropout_rate = 0.4
print(f"\nConstruyendo modelo LSTM para Fase 1")

model = Sequential(name="LSTM_Emotion_Classifier_Emb_Fase1")
model.add(Input(shape=(maxlen,), name="Input"))

# --- Capa Embedding CONGELADA ---
if use_pretrained and embedding_matrix is not None:
    print("Usando capa Embedding con pesos pre-entrenados (CONGELADOS).")
    model.add(Embedding(
        input_dim=vocab_size, output_dim=embedding_dim_model,
        weights=[embedding_matrix], 
        trainable=False, # <--- CONGELADO para Fase 1
        name="Pretrained_Embedding"
    ))
else: # Fallback por si acaso
    print(f"ERROR: No se pudieron usar embeddings pre-entrenados. Usando entrenables."); embedding_dim_model=128
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim_model, input_length=maxlen, trainable=True, name="Trainable_Embedding"))

# --- Resto del Modelo ---
# Probar con Bidireccional aquí si se desea: model.add(Bidirectional(LSTM(...)))
model.add(LSTM(units=hidden_units, name="LSTM_Layer"))
model.add(Dropout(dropout_rate, name="Dropout_LSTM"))
model.add(Dense(units=dense_units, activation='relu', name="Dense_Layer"))
model.add(Dense(units=num_classes, activation='softmax', name="Output_Softmax"))

In [ ]:
# 🚀 12. Compilar el modelo (FASE 1 - LR Moderada)
learning_rate_fase1 = 0.001 # Tasa de aprendizaje para Fase 1
optimizer_fase1 = Adam(learning_rate=learning_rate_fase1)
model.compile(loss='categorical_crossentropy', optimizer=optimizer_fase1, metrics=['accuracy'])
print(f"\nModelo compilado para Fase 1 con LR={learning_rate_fase1}")
model.summary()

In [ ]:
# 🚀 13. Callbacks (FASE 1)
checkpoint_path_fase1 = 'best_model_emociones_emb_fase1.keras'
model_checkpoint_fase1 = ModelCheckpoint(checkpoint_path_fase1, monitor='val_loss', save_best_only=True, verbose=1)
# Usaremos los mismos ES y ReduceLR, pero podrían definirse distintos si se quisiera
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=False) # False aquí, cargaremos el mejor explícitamente
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6, verbose=1) # Aumentar min_lr un poco
callbacks_list_fase1 = [model_checkpoint_fase1, early_stopping, reduce_lr]
print(f"Callbacks definidos para Fase 1. Checkpoint: {checkpoint_path_fase1}")

In [ ]:
# 🚀 14. Entrenar el modelo (FASE 1)
epochs_fase1 = 50 # EarlyStopping definirá el número real
batch_size = 128
print("\n--- Iniciando Entrenamiento FASE 1 (Embeddings Congelados) ---")
history_fase1 = model.fit(
  X_train, y_train,
  epochs=epochs_fase1, batch_size=batch_size, validation_data=(X_val, y_val),
  callbacks=callbacks_list_fase1, verbose=1
)
print("--- Entrenamiento FASE 1 Finalizado ---")
last_epoch_fase1 = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else epochs_fase1 -1 # Guardar última época real

In [ ]:
# 🚀 14b. Preparar para FASE 2 (Fine-tuning)
print("\n--- Preparando FASE 2 (Fine-tuning) ---")
model_loaded_fase2 = False
try:
    # Cargar el mejor modelo de la Fase 1
    if os.path.exists(checkpoint_path_fase1):
        model = load_model(checkpoint_path_fase1) # Cargar explícitamente el mejor
        print(f"Mejor modelo de Fase 1 cargado desde {checkpoint_path_fase1}.")

        # Buscar la capa Embedding y hacerla entrenable
        embedding_layer = model.get_layer(name='Pretrained_Embedding')
        if embedding_layer:
            embedding_layer.trainable = True # <--- DESCONGELAR
            print(f"Capa Embedding '{embedding_layer.name}' DESCONGELADA (trainable=True).")
        else:
            print("ADVERTENCIA: No se encontró la capa Embedding por nombre 'Pretrained_Embedding'.")

        # Re-compilar con LR MUY BAJA
        learning_rate_fase2 = 5e-5 # Probar 1e-5 o 5e-5
        optimizer_fase2 = Adam(learning_rate=learning_rate_fase2)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer_fase2, metrics=['accuracy'])
        print(f"Modelo re-compilado para Fase 2 con LR={learning_rate_fase2}.")
        model.summary() # Verificar cambio en parámetros entrenables
        model_loaded_fase2 = True
    else:
        print(f"ERROR: No se encontró el archivo de checkpoint de Fase 1: {checkpoint_path_fase1}")

except Exception as e:
    print(f"Error al preparar Fase 2: {e}")

In [ ]:
# 🚀 14c. Callbacks (FASE 2)
if model_loaded_fase2:
    checkpoint_path_fase2 = 'best_model_emociones_emb_fase2.keras' # Nuevo nombre para Fase 2
    model_checkpoint_fase2 = ModelCheckpoint(checkpoint_path_fase2, monitor='val_loss', save_best_only=True, verbose=1)
    # Re-instanciar ES y ReduceLR para Fase 2 (resetea sus estados internos)
    early_stopping_fase2 = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True) # True ahora para el final
    reduce_lr_fase2 = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1) # Podría necesitar min_lr más bajo
    callbacks_list_fase2 = [model_checkpoint_fase2, early_stopping_fase2, reduce_lr_fase2]
    print(f"Callbacks definidos para Fase 2. Checkpoint: {checkpoint_path_fase2}")
else:
    print("Saltando definición de callbacks de Fase 2 porque el modelo no se cargó.")

In [ ]:
# 🚀 14d. Entrenar el modelo (FASE 2 - Fine-tuning)
history_fase2 = None # Inicializar
if model_loaded_fase2:
    epochs_fase2_extra = 30 # Cuántas épocas MÁS intentar entrenar
    epochs_total = last_epoch_fase1 + epochs_fase2_extra
    print(f"\n--- Iniciando Entrenamiento FASE 2 (Fine-tuning desde época {last_epoch_fase1 + 1}) ---")
    history_fase2 = model.fit(
      X_train, y_train,
      epochs=epochs_total,
      initial_epoch=last_epoch_fase1 + 1, # Empezar desde la siguiente época
      batch_size=batch_size, # Puedes probar un batch size más pequeño aquí (e.g., 64)
      validation_data=(X_val, y_val),
      callbacks=callbacks_list_fase2,
      verbose=1
    )
    print("--- Entrenamiento FASE 2 Finalizado ---")
else:
    print("Saltando entrenamiento de Fase 2 porque el modelo no se cargó/preparó.")

In [ ]:
# 🚀 15. Visualizar el historial de entrenamiento (Fase 2 o combinado)
print("\nGraficando historial...")
final_history = None
if history_fase2:
    print("Mostrando historial de FASE 2.")
    final_history = history_fase2.history
elif history_fase1:
     print("Mostrando historial de FASE 1 (Fase 2 no se ejecutó).")
     final_history = history_fase1.history

if final_history:
    plt.figure(figsize=(12, 5))
    # Precisión
    plt.subplot(1, 2, 1)
    if 'accuracy' in final_history: plt.plot(final_history['accuracy'], label='Acc (Train)')
    if 'val_accuracy' in final_history: plt.plot(final_history['val_accuracy'], label='Acc (Val)')
    plt.title('Precisión (Última Fase)'); plt.xlabel('Época (relativa a la fase)'); plt.legend()
    # Pérdida
    plt.subplot(1, 2, 2)
    if 'loss' in final_history: plt.plot(final_history['loss'], label='Loss (Train)')
    if 'val_loss' in final_history: plt.plot(final_history['val_loss'], label='Loss (Val)')
    plt.title('Pérdida (Última Fase)'); plt.xlabel('Época (relativa a la fase)'); plt.legend()
    plt.tight_layout(); plt.show()
else:
    print("No hay historial para graficar.")

In [ ]:
# 🚀 16. Evaluar el mejor modelo guardado (Idealmente de Fase 2)
print("\nEvaluando el MEJOR MODELO FINAL...")
# El objeto 'model' debería tener los mejores pesos de Fase 2 si restore_best_weights=True
# O podemos cargar el checkpoint de Fase 2 explícitamente para estar seguros
best_model_path_final = checkpoint_path_fase2 if 'checkpoint_path_fase2' in locals() and os.path.exists(checkpoint_path_fase2) else checkpoint_path_fase1

try:
    print(f"Cargando mejor modelo desde: {best_model_path_final}")
    if os.path.exists(best_model_path_final):
        best_model = load_model(best_model_path_final)
        loss, accuracy = best_model.evaluate(X_val, y_val, verbose=0)
        print(f"\nEvaluación (Mejor Modelo): Pérdida={loss:.4f}, Precisión={accuracy:.4f}")
        y_pred_prob = best_model.predict(X_val); y_pred = np.argmax(y_pred_prob, axis=1)
        y_val_labels = np.argmax(y_val, axis=1)
        print("\nInforme de Clasificación:"); print(classification_report(y_val_labels, y_pred, target_names=label_encoder.classes_, zero_division=0))
        print("\nMatriz de Confusión:"); cm = confusion_matrix(y_val_labels, y_pred)
        plt.figure(figsize=(10, 8)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
        plt.xlabel('Predicha'); plt.ylabel('Verdadera'); plt.title('Matriz de Confusión'); plt.show()
    else:
        print("El archivo del mejor modelo no existe.")
except Exception as e: print(f"\nError evaluando: {e}")

In [ ]:
# 🚀 17. Función de Predicción y Ejemplos
# (Usa el 'best_model' cargado en la celda anterior)
def predecir_emocion(texto, modelo_pred, tokenizer_func, label_encoder_func, maxlen_func):
  if not texto or not isinstance(texto, str): return "Texto inválido", 0.0
  texto_limpio = clean_text(texto)
  if not texto_limpio: return "Texto vacío después de limpiar", 0.0
  secuencia = tokenizer_func.texts_to_sequences([texto_limpio])
  padded_secuencia = pad_sequences(secuencia, maxlen=maxlen_func, padding='post', truncating='post')
  prediccion_prob = modelo_pred.predict(padded_secuencia)
  etiqueta_idx = np.argmax(prediccion_prob[0])
  probabilidad = np.max(prediccion_prob[0])
  etiqueta_predicha = label_encoder_func.inverse_transform([etiqueta_idx])[0]
  return etiqueta_predicha, probabilidad

if 'best_model' in locals(): # Verificar si se cargó el mejor modelo
    predictor_model = best_model
    print("\nProbando predicción con el mejor modelo cargado:")
    # Ejemplos...
    texto_prueba = "alguna vez se deprimen, porque la única persona con la que quieren hablar y pasar el tiempo está castigada... así que solo están sentados allí, sin hacer nada, porque no tienen ganas de hablar o pasar tiempo con nadie más."
    emocion_predicha, prob = predecir_emocion(texto_prueba, predictor_model, tokenizer, label_encoder, maxlen)
    print(f"Texto: '{texto_prueba}'\nEmoción Predicha: {emocion_predicha} (Probabilidad: {prob:.4f})\n")
    texto_prueba2 = "me siento muy mal, no quiero vivir"
    emocion_predicha2, prob2 = predecir_emocion(texto_prueba2, predictor_model, tokenizer, label_encoder, maxlen)
    print(f"Texto: '{texto_prueba2}'\nEmoción Predicha: {emocion_predicha2} (Probabilidad: {prob2:.4f})\n")
    texto_prueba3 = "Estoy muy feliz con los resultados del examen"
    emocion_predicha3, prob3 = predecir_emocion(texto_prueba3, predictor_model, tokenizer, label_encoder, maxlen)
    print(f"Texto: '{texto_prueba3}'\nEmoción Predicha: {emocion_predicha3} (Probabilidad: {prob3:.4f})\n")
    texto_prueba4 = "El día estuvo normal, nada especial ocurrió, que aburrido."
    emocion_predicha4, prob4 = predecir_emocion(texto_prueba4, predictor_model, tokenizer, label_encoder, maxlen)
    print(f"Texto: '{texto_prueba4}'\nEmoción Predicha: {emocion_predicha4} (Probabilidad: {prob4:.4f})\n")
else:
    print("\nNo se pudo encontrar un modelo ('best_model') para realizar predicciones.")

In [1]:
# 🚀 18. Guardar artefactos finales
# (Código igual, guarda tokenizer y encoder)
try:
  with open('tokenizer_emociones.pickle', 'wb') as h: pickle.dump(tokenizer, h, protocol=pickle.HIGHEST_PROTOCOL)
  print("Tokenizer guardado.")
except Exception as e: print(f"Error guardando tokenizer: {e}")
try:
  with open('label_encoder_emociones.pickle', 'wb') as h: pickle.dump(label_encoder, h, protocol=pickle.HIGHEST_PROTOCOL)
  print("Label Encoder guardado.")
except Exception as e: print(f"Error guardando label encoder: {e}")

print(f"\nMejor modelo Fase 1 guardado como: {checkpoint_path_fase1}")
if 'checkpoint_path_fase2' in locals(): print(f"Mejor modelo Fase 2 guardado como: {checkpoint_path_fase2}")

Error guardando tokenizer: name 'pickle' is not defined
Error guardando label encoder: name 'pickle' is not defined


NameError: name 'checkpoint_path_fase1' is not defined